In [ ]:
import os
import pyspark
conf = pyspark.SparkConf()
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','8g')
sc = pyspark.SparkContext(conf=conf)
sc.setLogLevel("OFF")
spark = pyspark.SQLContext.getOrCreate(sc)

In [2]:
#rm -r outputs

# Data processing

In [70]:
from pyspark.sql.functions import col, udf, when, lit, split, countDistinct, explode
from pyspark.sql.types import StringType, FloatType, IntegerType

import re
from nltk.corpus import stopwords
import nltk
#nltk.download('stopwords')

ImportError: cannot import name 'vector_to_array' from 'pyspark.sql.functions' (/opt/conda/envs/bigdata-fall22/lib/python3.7/site-packages/pyspark/sql/functions.py)

In [4]:
# load datasets
train_data_path = "data/train.csv"
df_train = spark.read.option("inferSchema", "true") \
                     .option("delimiter", ",") \
                     .option("header", True) \
                     .csv(train_data_path)

In [5]:
df_train.describe()

summary,id,label,tweet
count,31962,31962,31962
mean,15981.5,0.07014579813528565,null
stddev,9226.778988357746,0.2553965659892674,null
min,1,0,#friday #beat...
max,31962,1,ó¾­© duchess ó¾...


In [6]:
df_train.show()

+---+-----+--------------------+
| id|label|               tweet|
+---+-----+--------------------+
|  1|    0| @user when a fat...|
|  2|    0|@user @user thank...|
|  3|    0|  bihday your maj...|
|  4|    0|#model   i love u...|
|  5|    0| factsguide: soci...|
|  6|    0|[2/2] huge fan fa...|
|  7|    0| @user camping to...|
|  8|    0|the next school y...|
|  9|    0|we won!!! love th...|
| 10|    0| @user @user welc...|
| 11|    0| â #ireland con...|
| 12|    0|we are so selfish...|
| 13|    0|i get to see my d...|
| 14|    1|@user #cnn calls ...|
| 15|    1|no comment!  in #...|
| 16|    0|ouch...junior is ...|
| 17|    0|i am thankful for...|
| 18|    1|retweet if you ag...|
| 19|    0|its #friday! ð...|
| 20|    0|as we all know, e...|
+---+-----+--------------------+
only showing top 20 rows



In [7]:
# check duplicate samples
traindf_duplicates = df_train.groupBy("tweet", "label").count().filter(col("count") > 1)
if traindf_duplicates.count()>0:
    print("Total number of duplicates in train data:", traindf_duplicates.agg({'count': 'sum'}).collect()[0][0])
else:
    print("No duplicates in train data")

Total number of duplicates in train data: 3130


In [8]:
# remove duplicates
df_train = df_train.dropDuplicates(["label", "tweet"])
print("Distinct count: "+ str(df_train.count()))

Distinct count: 29528


In [9]:
df_train.show()

+----+-----+--------------------+
|  id|label|               tweet|
+----+-----+--------------------+
| 281|    0|last day at work ...|
| 858|    0|2 hours to go unt...|
| 975|    0| @user omg bitch ...|
|1133|    0|two days ... #17 ...|
|1232|    0|to the best dad i...|
|1289|    0|omg!!! luckily we...|
|1577|    0|i get to the sara...|
|1701|    0| @user billy goat...|
|1849|    0| â #boe holds i...|
|2064|    0| @user our bassis...|
|2569|    0|always be   #with...|
|2625|    0|sad.wav #suga #kp...|
|2736|    0|let the #getfat r...|
|2871|    0|mps &amp; senator...|
|3653|    0|  #makaveli #day ...|
|3703|    0|pretty happy with...|
|3813|    0|just listened to ...|
|4158|    0|you never knew th...|
|4342|    0|love watching the...|
|4411|    0|love these beauti...|
+----+-----+--------------------+
only showing top 20 rows



In [10]:
# remove stopwords and special char
nltk.data.path.append("nltk_data")
stopwords_list = stopwords.words('english')
stopwords_list.extend(["amp"])

@udf(returnType=StringType())
def get_cleaned_tweet(tweet):
    tweet = re.sub(r'http\S+', '', tweet) # remove urls
    tweet = re.sub(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+', '', tweet) # remove emails
    tweet = re.sub('[^a-zA-Z0-9]', " ", tweet) # replace special chars with white space
    tweet = re.sub(r'\s+', ' ', tweet) # replace whitespace(s) with a single space
    tweet = re.sub(r'^\s+|\s+?$', '', tweet) # remove leading and trailing whitespace
    tweet = re.sub(r'\d+(\.\d+)?', '[num]', tweet) # replace all numbers with string `num`
    tweet = tweet.lower() # lowercase
    tweet = ' '.join([word for word in tweet.split() if not word in stopwords_list])
    
    return tweet

df_train = df_train.withColumn("tweet", get_cleaned_tweet("tweet"))
df_train.show()

+----+-----+--------------------+
|  id|label|               tweet|
+----+-----+--------------------+
| 281|    0|last day work tom...|
| 858|    0|[num] hours go mu...|
| 975|    0|user omg bitch kn...|
|1133|    0|two days [num] da...|
|1232|    0|best dad world fa...|
|1289|    0|omg luckily got g...|
|1577|    0|get sarajevo int ...|
|1701|    0|user billy goat t...|
|1849|    0|boe holds fire an...|
|2064|    0|user bassist left...|
|2569|    0|     always withboby|
|2625|    0|sad wav suga kpop...|
|2736|    0|let getfat reunio...|
|2871|    0|mps senators get ...|
|3653|    0|makaveli day happ...|
|3703|    0|pretty happy smil...|
|3813|    0|listened savannah...|
|4158|    0|never knew day wo...|
|4342|    0|love watching iom...|
|4411|    0|love beautiful la...|
+----+-----+--------------------+
only showing top 20 rows



### get number of distinct words in the dataset, that will give us hint on number of features to use for BOW

In [11]:
# split by space(s) then explode to get words
df_train_temp = df_train.withColumn("tweet", explode(split("tweet", "\s+"))).withColumnRenamed("tweet", "unigram")
df_train_temp.show()

+---+-----+---------------+
| id|label|        unigram|
+---+-----+---------------+
|281|    0|           last|
|281|    0|            day|
|281|    0|           work|
|281|    0|       tomorrow|
|281|    0|           self|
|281|    0|       employed|
|281|    0|         monday|
|281|    0|         scared|
|281|    0|redditchbizhour|
|858|    0|          [num]|
|858|    0|          hours|
|858|    0|             go|
|858|    0|          music|
|858|    0|          video|
|858|    0|       released|
|858|    0|          music|
|858|    0|      edinburgh|
|858|    0|       scotland|
|858|    0|           band|
|858|    0|           user|
+---+-----+---------------+
only showing top 20 rows



In [12]:
# count number of distinct words
total_words_count = df_train_temp.select("unigram").count()
distinct_words_count = df_train_temp.select("unigram").distinct().count()
print(f"Total number of words: {total_words_count}\nDistinct words count: {distinct_words_count}")

Total number of words: 250447
Distinct words count: 37927


# Spark Data Transformation

In [13]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [14]:
# tokenization
tokenizer = Tokenizer(inputCol="tweet", outputCol="words")
wordsData = tokenizer.transform(df_train)

# term frequency
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures") # default numFeatures: int=262144 to avoid collision
featurizedData = hashingTF.transform(wordsData)

# inverse document frequency
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

rescaledData.select("label", "features").show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|(262144,[5381,291...|
|    0|(262144,[14118,56...|
|    0|(262144,[991,8229...|
|    0|(262144,[4017,246...|
|    0|(262144,[60080,80...|
|    0|(262144,[5569,100...|
|    0|(262144,[1303,850...|
|    0|(262144,[15377,24...|
|    0|(262144,[20785,21...|
|    0|(262144,[25491,65...|
|    0|(262144,[39081,79...|
|    0|(262144,[23837,45...|
|    0|(262144,[39965,17...|
|    0|(262144,[30020,42...|
|    0|(262144,[69959,73...|
|    0|(262144,[23071,77...|
|    0|(262144,[18545,85...|
|    0|(262144,[991,8538...|
|    0|(262144,[20719,28...|
|    0|(262144,[36114,43...|
+-----+--------------------+
only showing top 20 rows



In [15]:
# since there are class imbalance, add class weight
hate_df = rescaledData.filter("label == 1")
hate_ratio = hate_df.count() / df_train.count()
hate_class_weight = 1 - hate_ratio
neg_class_weight = hate_ratio
print("Hate Class Weight:", hate_class_weight)
print("Not Hate Class Weight:", neg_class_weight)

Hate Class Weight: 0.9318274180438906
Not Hate Class Weight: 0.06817258195610945


In [16]:
# add weight column
rescaledData = rescaledData.withColumn(
    "weight", 
    when(df_train.label == 1, hate_class_weight).otherwise(neg_class_weight)
)

# add no_weight column
rescaledData = rescaledData.withColumn(
    "no_weight",
    lit(1.0)
)

In [17]:
rescaledData.show()

+----+-----+--------------------+--------------------+--------------------+--------------------+-------------------+---------+
|  id|label|               tweet|               words|         rawFeatures|            features|             weight|no_weight|
+----+-----+--------------------+--------------------+--------------------+--------------------+-------------------+---------+
| 281|    0|last day work tom...|[last, day, work,...|(262144,[5381,291...|(262144,[5381,291...|0.06817258195610945|      1.0|
| 858|    0|[num] hours go mu...|[[num], hours, go...|(262144,[14118,56...|(262144,[14118,56...|0.06817258195610945|      1.0|
| 975|    0|user omg bitch kn...|[user, omg, bitch...|(262144,[991,8229...|(262144,[991,8229...|0.06817258195610945|      1.0|
|1133|    0|two days [num] da...|[two, days, [num]...|(262144,[4017,246...|(262144,[4017,246...|0.06817258195610945|      1.0|
|1232|    0|best dad world fa...|[best, dad, world...|(262144,[60080,80...|(262144,[60080,80...|0.0681725819561

# Train LogisticRegression

In [21]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

In [22]:
# generate the train/test split.
(train, test) = rescaledData.randomSplit([0.8, 0.2], seed=124312)

In [23]:
def train_model(train, test, maxIter=1000, regParam=1.0, family="auto", elasticNetParam=0,
                weightCol=None, output_path="ouputs/", store_model=True):
    # instantiate the base classifier.
    lr = LogisticRegression(maxIter=maxIter, tol=1E-6, regParam=regParam, 
                            weightCol=weightCol, family=family, elasticNetParam=elasticNetParam)

    # train the multiclass model.
    lrModel = lr.fit(train)

    # score the model on test data.
    predictions = lrModel.transform(test)
    
    # store model checkpoint
    if store_model:
        lrModel.write().save(output_path)
                
    return lrModel, predictions

In [24]:
def eval_model(predictions):
    # Create evaluators for overall f1-score, recall and precision
    f1_evaluator = MulticlassClassificationEvaluator(metricName="f1")
    precision_evaluator = MulticlassClassificationEvaluator(metricName="weightedPrecision")
    recall_evaluator = MulticlassClassificationEvaluator(metricName="weightedRecall")

    # Compute the F1 score, precision, and recall for each class
    f1 = f1_evaluator.evaluate(predictions)
    precision = precision_evaluator.evaluate(predictions)
    recall = recall_evaluator.evaluate(predictions)


    # compute class-wise f1-score, recall, precision and overall confusion matrix
    #important: need to cast to float type, and order by prediction, else it won't work
    preds_and_labels = predictions.select(['prediction','label']).withColumn('label', col('label').cast(FloatType())).orderBy('prediction')

    #select only prediction and label columns
    preds_and_labels = preds_and_labels.select(['prediction','label'])

    metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))

    print(f"Confusion Matrix:\n{metrics.confusionMatrix().toArray()}")

    print(f"\nOverall F1-score: {f1}")
    print(f"Overall Recall: {recall}")
    print(f"Overall Precision: {precision}")

    print(f"\nClass=1(Hate) F1-score: {metrics.fMeasure(label=1.0)}")
    print(f"Class=1(Hate) Recall: {metrics.recall(label=1.0)}")
    print(f"Class=1(Hate) Precision: {metrics.precision(label=1.0)}")

    print(f"\nClass=0(Not Hate) F1-score: {metrics.fMeasure(label=0.0)}")
    print(f"Class=0(Not Hate) Recall: {metrics.recall(label=0.0)}")
    print(f"Class=0(Not Hate) Precision: {metrics.precision(label=0.0)}")
    

In [60]:
# baseline training
maxIter = 1000
regParam = 1.0
weightCol = "no_weight"
output_path = "outputs/lr_no_weight_no_tune/"

# train model
model, predictions = train_model(train, test, maxIter, regParam=regParam, 
                                 weightCol=weightCol, output_path=output_path)

# evaluate on testset
eval_model(predictions)

Confusion Matrix:
[[5526.    0.]
 [ 376.   17.]]

Overall F1-score: 0.9083925921908943
Overall Recall: 0.9364757560398715
Overall Precision: 0.9405227088912793

Class=1(Hate) F1-score: 0.08292682926829269
Class=1(Hate) Recall: 0.043256997455470736
Class=1(Hate) Precision: 1.0

Class=0(Not Hate) F1-score: 0.9670983549177459
Class=0(Not Hate) Recall: 1.0
Class=0(Not Hate) Precision: 0.9362927821077601


In [61]:
predictions.select(["label", "prediction", "probability", "rawPrediction"]).show()

+-----+----------+--------------------+--------------------+
|label|prediction|         probability|       rawPrediction|
+-----+----------+--------------------+--------------------+
|    0|       0.0|[0.94755957982777...|[2.89421214476344...|
|    0|       0.0|[0.94502140295065...|[2.84426361092355...|
|    0|       0.0|[0.94411399171478...|[2.82693286297164...|
|    0|       0.0|[0.95683264418134...|[3.09854394344199...|
|    0|       0.0|[0.94836478892555...|[2.91053540175152...|
|    0|       0.0|[0.95300873521270...|[3.00966234078035...|
|    1|       0.0|[0.84911887500526...|[1.72770691910474...|
|    0|       0.0|[0.91053557209362...|[2.32019187443577...|
|    0|       0.0|[0.92918355548152...|[2.57421506229817...|
|    0|       0.0|[0.94284842821026...|[2.80319864336407...|
|    0|       0.0|[0.96329298053091...|[3.26738959987974...|
|    0|       0.0|[0.95274422943442...|[3.00377170813416...|
|    0|       0.0|[0.95763488121803...|[3.11814122582420...|
|    0|       0.0|[0.948

# Hyperparameter Tuning

In [62]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

### tune with different hyperparamters

In [63]:
# baseline training
maxIter = 1000
regParams = [0.001, 0.003, 0.01, 0.03, 0.1, 0.3]
families = ["binomial", "multinomial"]
weightCols = ["no_weight", "weight"]
output_path = "outputs/lr_no_weight_no_tune/"
elasticNetParams = [0.001, 0.003, 0.01, 0.015, 0.02]

for weightCol in weightCols:
    for regParam in regParams:
        for family in families:
            for elasticNetParam in elasticNetParams:
                print("\n", "*"*20)
                print(f"regParam: {regParam}, weightCol: {weightCol}, family: {family}, elasticNetParam: {elasticNetParam}")

                # train model
                _, predictions = train_model(train, test, maxIter, 
                                             regParam=regParam, weightCol=weightCol, 
                                             family=family, elasticNetParam=elasticNetParam,
                                             output_path=output_path, store_model=False)

                # eval model on devset
                eval_model(predictions)


 ********************
regParam: 0.001, weightCol: no_weight, family: binomial, elasticNetParam: 0.001


Confusion Matrix:
[[5434.   92.]
 [ 189.  204.]]

Overall F1-score: 0.9493904849665813
Overall Recall: 0.9525257644872445
Overall Precision: 0.9479830563592146

Class=1(Hate) F1-score: 0.5921625544267054
Class=1(Hate) Recall: 0.5190839694656488
Class=1(Hate) Precision: 0.6891891891891891

Class=0(Not Hate) F1-score: 0.9747959458247376
Class=0(Not Hate) Recall: 0.9833514296055013
Class=0(Not Hate) Precision: 0.9663880490841188

 ********************
regParam: 0.001, weightCol: no_weight, family: binomial, elasticNetParam: 0.003


Confusion Matrix:
[[5436.   90.]
 [ 192.  201.]]

Overall F1-score: 0.9490223157822798
Overall Recall: 0.9523568170299037
Overall Precision: 0.9476150282790815

Class=1(Hate) F1-score: 0.5877192982456141
Class=1(Hate) Recall: 0.5114503816793893
Class=1(Hate) Precision: 0.6907216494845361

Class=0(Not Hate) F1-score: 0.9747175901022055
Class=0(Not Hate) Recall: 0.9837133550488599
Class=0(Not Hate) Precision: 0.9658848614072495

 ********************
regParam: 0.001, weightCol: no_weight, family: binomial, elasticNetParam: 0.01


Confusion Matrix:
[[5429.   97.]
 [ 190.  203.]]

Overall F1-score: 0.9484608662006904
Overall Recall: 0.9515120797431998
Overall Precision: 0.9469631166864072

Class=1(Hate) F1-score: 0.5858585858585859
Class=1(Hate) Recall: 0.5165394402035624
Class=1(Hate) Precision: 0.6766666666666666

Class=0(Not Hate) F1-score: 0.9742485419470615
Class=0(Not Hate) Recall: 0.9824466159971046
Class=0(Not Hate) Precision: 0.9661861541199501

 ********************
regParam: 0.001, weightCol: no_weight, family: binomial, elasticNetParam: 0.015


Confusion Matrix:
[[5429.   97.]
 [ 190.  203.]]

Overall F1-score: 0.9484608662006904
Overall Recall: 0.9515120797431998
Overall Precision: 0.9469631166864072

Class=1(Hate) F1-score: 0.5858585858585859
Class=1(Hate) Recall: 0.5165394402035624
Class=1(Hate) Precision: 0.6766666666666666

Class=0(Not Hate) F1-score: 0.9742485419470615
Class=0(Not Hate) Recall: 0.9824466159971046
Class=0(Not Hate) Precision: 0.9661861541199501

 ********************
regParam: 0.001, weightCol: no_weight, family: binomial, elasticNetParam: 0.02


...

Confusion Matrix:
[[5324.  202.]
 [ 140.  253.]]

Overall F1-score: 0.9441692284698652
Overall Recall: 0.9422199695894576
Overall Precision: 0.946601908249747

Class=1(Hate) F1-score: 0.5966981132075473
Class=1(Hate) Recall: 0.6437659033078881
Class=1(Hate) Precision: 0.5560439560439561

Class=0(Not Hate) F1-score: 0.9688808007279345
Class=0(Not Hate) Recall: 0.9634455302207745
Class=0(Not Hate) Precision: 0.9743777452415813

 ********************
regParam: 0.3, weightCol: weight, family: multinomial, elasticNetParam: 0.003


Confusion Matrix:
[[5362.  164.]
 [ 140.  253.]]

Overall F1-score: 0.9493449753948351
Overall Recall: 0.9486399729684069
Overall Precision: 0.9501314684740603

Class=1(Hate) F1-score: 0.6246913580246914
Class=1(Hate) Recall: 0.6437659033078881
Class=1(Hate) Precision: 0.6067146282973621

Class=0(Not Hate) F1-score: 0.9724338048603556
Class=0(Not Hate) Recall: 0.9703221136445892
Class=0(Not Hate) Precision: 0.9745547073791349

 ********************
regParam: 0.3, weightCol: weight, family: multinomial, elasticNetParam: 0.01


Confusion Matrix:
[[5328.  198.]
 [ 133.  260.]]

Overall F1-score: 0.9460486397868486
Overall Recall: 0.9440783916202061
Overall Precision: 0.9485584377906028

Class=1(Hate) F1-score: 0.6110458284371327
Class=1(Hate) Recall: 0.6615776081424937
Class=1(Hate) Precision: 0.5676855895196506

Class=0(Not Hate) F1-score: 0.9698734868480932
Class=0(Not Hate) Recall: 0.9641693811074918
Class=0(Not Hate) Precision: 0.9756454861746933

 ********************
regParam: 0.3, weightCol: weight, family: multinomial, elasticNetParam: 0.015


Confusion Matrix:
[[5271.  255.]
 [ 120.  273.]]

Overall F1-score: 0.9408962800917184
Overall Recall: 0.9366447034972124
Overall Precision: 0.9471521972105286

Class=1(Hate) F1-score: 0.5928338762214984
Class=1(Hate) Recall: 0.6946564885496184
Class=1(Hate) Precision: 0.5170454545454546

Class=0(Not Hate) F1-score: 0.9656499038197307
Class=0(Not Hate) Recall: 0.9538545059717698
Class=0(Not Hate) Precision: 0.9777406789092933

 ********************
regParam: 0.3, weightCol: weight, family: multinomial, elasticNetParam: 0.02


Confusion Matrix:
[[5213.  313.]
 [ 112.  281.]]

Overall F1-score: 0.9348435213436888
Overall Recall: 0.9281973306301741
Overall Precision: 0.9453770122960532

Class=1(Hate) F1-score: 0.569402228976697
Class=1(Hate) Recall: 0.7150127226463104
Class=1(Hate) Precision: 0.4730639730639731

Class=0(Not Hate) F1-score: 0.9608331029398213
Class=0(Not Hate) Recall: 0.9433586681143684
Class=0(Not Hate) Precision: 0.9789671361502348


In [25]:
# best-model training
maxIter = 1000
regParam = 0.1
weightCol = "weight"
family = "binomial"
elasticNetParam = 0.015
output_path = "outputs/lr_bestModel/"

# train model
_, predictions = train_model(train, test, maxIter, 
                             regParam=regParam, weightCol=weightCol, 
                             family=family, elasticNetParam=elasticNetParam,
                             output_path=output_path, store_model=True)

# evaluate on testset
eval_model(predictions)

/opt/conda/envs/bigdata-fall22/lib/python3.7/site-packages/pyspark/sql/context.py:159: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


Confusion Matrix:
[[5377.  149.]
 [ 141.  252.]]

Overall F1-score: 0.9512343170756914
Overall Recall: 0.9510052373711776
Overall Precision: 0.9514729087154187

Class=1(Hate) F1-score: 0.6347607052896725
Class=1(Hate) Recall: 0.6412213740458015
Class=1(Hate) Precision: 0.628428927680798

Class=0(Not Hate) F1-score: 0.9737413980441868
Class=0(Not Hate) Recall: 0.9730365544697792
Class=0(Not Hate) Precision: 0.9744472635012685


# Inference

In [27]:
from pyspark.ml.classification import LogisticRegressionModel

In [28]:
model = LogisticRegressionModel.load("outputs/lr_bestModel/")

In [29]:
# generate the train/test split.
(train, test) = rescaledData.randomSplit([0.8, 0.2], seed=124312)

In [44]:
# score the model on test data.
predictions = model.transform(test)

In [45]:
predictions.select(["tweet", "label", "prediction", "probability", "rawPrediction"]).show()

+--------------------+-----+----------+--------------------+--------------------+
|               tweet|label|prediction|         probability|       rawPrediction|
+--------------------+-----+----------+--------------------+--------------------+
|      bihday majesty|    0|       0.0|[0.83404860397635...|[1.61459672836432...|
|factsguide societ...|    0|       0.0|[0.85597177129278...|[1.78222808508314...|
|user camping tomo...|    0|       0.0|[0.75729942617892...|[1.13793024134758...|
|lovely echeveria ...|    0|       0.0|[0.92058284991261...|[2.45029266096771...|
|feeling blue illu...|    0|       0.0|[0.82884620388908...|[1.57747207313392...|
|happy work confer...|    0|       0.0|[0.93242040609400...|[2.62447772029726...|
|user hey white pe...|    1|       1.0|[0.11672056050329...|[-2.0238589102385...|
|user user might s...|    0|       0.0|[0.58387112426786...|[0.33868527113485...|
|carrying gun help...|    0|       0.0|[0.81160025372916...|[1.46044190519634...|
|[num] things in

In [46]:
# check probabilities for class=1
predictions.where("label = 1").select(["tweet", "label", "prediction", "probability", "rawPrediction"]).show()

+--------------------+-----+----------+--------------------+--------------------+
|               tweet|label|prediction|         probability|       rawPrediction|
+--------------------+-----+----------+--------------------+--------------------+
|user hey white pe...|    1|       1.0|[0.11672056050329...|[-2.0238589102385...|
|smaller hands sho...|    1|       0.0|[0.90787002510481...|[2.28790087357002...|
|stereotyping prej...|    1|       0.0|[0.61730318180139...|[0.47811720546150...|
|many pols passed ...|    1|       1.0|[0.19652223588256...|[-1.4081739263992...|
|user dying light ...|    1|       1.0|[0.13519454022882...|[-1.8557897997684...|
|user biherism abs...|    1|       1.0|[0.02288524279828...|[-3.7541118203542...|
|carolyn cooper ug...|    1|       1.0|[0.35114613221755...|[-0.6140050582180...|
|standing racism h...|    1|       1.0|[0.19583891580500...|[-1.4125071394050...|
|user allahsoil vp...|    1|       1.0|[0.13242671110755...|[-1.8796706224453...|
|[num] jan inter

In [47]:
# filter out unneed columns
predictions = predictions.select(["tweet", "label", "prediction", "probability"])
predictions.show()

+--------------------+-----+----------+--------------------+
|               tweet|label|prediction|         probability|
+--------------------+-----+----------+--------------------+
|      bihday majesty|    0|       0.0|[0.83404860397635...|
|factsguide societ...|    0|       0.0|[0.85597177129278...|
|user camping tomo...|    0|       0.0|[0.75729942617892...|
|lovely echeveria ...|    0|       0.0|[0.92058284991261...|
|feeling blue illu...|    0|       0.0|[0.82884620388908...|
|happy work confer...|    0|       0.0|[0.93242040609400...|
|user hey white pe...|    1|       1.0|[0.11672056050329...|
|user user might s...|    0|       0.0|[0.58387112426786...|
|carrying gun help...|    0|       0.0|[0.81160025372916...|
|[num] things incr...|    0|       0.0|[0.74531950559030...|
|eur usd clears [n...|    0|       0.0|[0.97560182284792...|
|received dis user...|    0|       0.0|[0.92849196683802...|
|marvel song music...|    0|       0.0|[0.94358096380952...|
|spell brexit refe...|  

In [48]:
predictions = predictions.withColumnRenamed("probability", "confidence_score")
predictions.show()

+--------------------+-----+----------+--------------------+
|               tweet|label|prediction|    confidence_score|
+--------------------+-----+----------+--------------------+
|      bihday majesty|    0|       0.0|[0.83404860397635...|
|factsguide societ...|    0|       0.0|[0.85597177129278...|
|user camping tomo...|    0|       0.0|[0.75729942617892...|
|lovely echeveria ...|    0|       0.0|[0.92058284991261...|
|feeling blue illu...|    0|       0.0|[0.82884620388908...|
|happy work confer...|    0|       0.0|[0.93242040609400...|
|user hey white pe...|    1|       1.0|[0.11672056050329...|
|user user might s...|    0|       0.0|[0.58387112426786...|
|carrying gun help...|    0|       0.0|[0.81160025372916...|
|[num] things incr...|    0|       0.0|[0.74531950559030...|
|eur usd clears [n...|    0|       0.0|[0.97560182284792...|
|received dis user...|    0|       0.0|[0.92849196683802...|
|marvel song music...|    0|       0.0|[0.94358096380952...|
|spell brexit refe...|  

In [50]:
# change prediction to int
predictions = predictions.withColumn(
    "prediction", col("prediction").cast(IntegerType()).cast(StringType())
).withColumn(
    "label", col("label").cast(StringType())
)
predictions.show()

+--------------------+-----+----------+--------------------+
|               tweet|label|prediction|    confidence_score|
+--------------------+-----+----------+--------------------+
|      bihday majesty|    0|         0|[0.83404860397635...|
|factsguide societ...|    0|         0|[0.85597177129278...|
|user camping tomo...|    0|         0|[0.75729942617892...|
|lovely echeveria ...|    0|         0|[0.92058284991261...|
|feeling blue illu...|    0|         0|[0.82884620388908...|
|happy work confer...|    0|         0|[0.93242040609400...|
|user hey white pe...|    1|         1|[0.11672056050329...|
|user user might s...|    0|         0|[0.58387112426786...|
|carrying gun help...|    0|         0|[0.81160025372916...|
|[num] things incr...|    0|         0|[0.74531950559030...|
|eur usd clears [n...|    0|         0|[0.97560182284792...|
|received dis user...|    0|         0|[0.92849196683802...|
|marvel song music...|    0|         0|[0.94358096380952...|
|spell brexit refe...|  

In [52]:
# map label and prediction to hate | not_hate
mapping = {"0":"not_hate", "1":"hate"}
predictions = predictions.replace(to_replace=mapping, subset=["label", "prediction"])
predictions.show()

+--------------------+--------+----------+--------------------+
|               tweet|   label|prediction|    confidence_score|
+--------------------+--------+----------+--------------------+
|      bihday majesty|not_hate|  not_hate|[0.83404860397635...|
|factsguide societ...|not_hate|  not_hate|[0.85597177129278...|
|user camping tomo...|not_hate|  not_hate|[0.75729942617892...|
|lovely echeveria ...|not_hate|  not_hate|[0.92058284991261...|
|feeling blue illu...|not_hate|  not_hate|[0.82884620388908...|
|happy work confer...|not_hate|  not_hate|[0.93242040609400...|
|user hey white pe...|    hate|      hate|[0.11672056050329...|
|user user might s...|not_hate|  not_hate|[0.58387112426786...|
|carrying gun help...|not_hate|  not_hate|[0.81160025372916...|
|[num] things incr...|not_hate|  not_hate|[0.74531950559030...|
|eur usd clears [n...|not_hate|  not_hate|[0.97560182284792...|
|received dis user...|not_hate|  not_hate|[0.92849196683802...|
|marvel song music...|not_hate|  not_hat

In [74]:
# type of confidence
predictions.schema

StructType([StructField('tweet', StringType(), True), StructField('label', StringType(), True), StructField('prediction', StringType(), True), StructField('confidence_score', VectorUDT(), True)])

In [75]:
# change DenseVector to Array
from pyspark.ml.functions import vector_to_array
predictions = predictions.withColumn("confidence_score", vector_to_array("confidence_score"))
predictions.schema

StructType([StructField('tweet', StringType(), True), StructField('label', StringType(), True), StructField('prediction', StringType(), True), StructField('confidence_score', ArrayType(DoubleType(), False), False)])

In [80]:
# filter out confidence_score to only keep the one correspond to predicted class
@udf(returnType=FloatType())
def get_cls_confidence(prediction, confidence_score):
    if prediction == "not_hate":
        return confidence_score[0]
    else:
        return confidence_score[1]
    
predictions = predictions.withColumn("confidence_score", get_cls_confidence("prediction", "confidence_score"))
predictions.show()

+--------------------+--------+----------+----------------+
|               tweet|   label|prediction|confidence_score|
+--------------------+--------+----------+----------------+
|      bihday majesty|not_hate|  not_hate|       0.8340486|
|factsguide societ...|not_hate|  not_hate|      0.85597175|
|user camping tomo...|not_hate|  not_hate|       0.7572994|
|lovely echeveria ...|not_hate|  not_hate|      0.92058283|
|feeling blue illu...|not_hate|  not_hate|       0.8288462|
|happy work confer...|not_hate|  not_hate|      0.93242043|
|user hey white pe...|    hate|      hate|      0.88327944|
|user user might s...|not_hate|  not_hate|       0.5838711|
|carrying gun help...|not_hate|  not_hate|      0.81160027|
|[num] things incr...|not_hate|  not_hate|       0.7453195|
|eur usd clears [n...|not_hate|  not_hate|      0.97560185|
|received dis user...|not_hate|  not_hate|      0.92849195|
|marvel song music...|not_hate|  not_hate|        0.943581|
|spell brexit refe...|not_hate|  not_hat

In [88]:
a = list(map(lambda row: row.asDict(), predictions.collect()))
a

[{'tweet': 'bihday majesty',
  'label': 'not_hate',
  'prediction': 'not_hate',
  'confidence_score': 0.8340486288070679},
 {'tweet': 'factsguide society motivation',
  'label': 'not_hate',
  'prediction': 'not_hate',
  'confidence_score': 0.8559717535972595},
 {'tweet': 'user camping tomorrow user user user user user user user danny',
  'label': 'not_hate',
  'prediction': 'not_hate',
  'confidence_score': 0.7572994232177734},
 {'tweet': 'lovely echeveria blooms flowers grow gardening iphonesia bliss blooms basilicabotanica',
  'label': 'not_hate',
  'prediction': 'not_hate',
  'confidence_score': 0.9205828309059143},
 {'tweet': 'feeling blue illustration',
  'label': 'not_hate',
  'prediction': 'not_hate',
  'confidence_score': 0.8288462162017822},
 {'tweet': 'happy work conference right mindset leads culture development organizations work mindset',
  'label': 'not_hate',
  'prediction': 'not_hate',
  'confidence_score': 0.9324204325675964},
 {'tweet': 'user hey white people call peo

{'tweet': 'bihday majesty', 'label': 'not_hate', 'prediction': 'not_hate', 'confidence_score': 0.8340486288070679}
